In [ ]:
%load_ext autoreload
%autoreload 2

import os,sys
sys.path.append(os.path.abspath(".."))
sys.path.append(os.path.abspath("../.."))
sys.path.append(os.path.abspath("../training"))

In [ ]:
import os
os.environ["EXP_NAME"] = "movie_recom"
os.environ["MAX_RATING"] = "5"
os.environ["MLFLOW_TRACKING_URI"] = "http://localhost:8081"
os.environ["EPOCH"] = "2"

os.environ["AWS_DEFAULT_REGION"] = "us-east-1"
os.environ["AWS_ACCESS_KEY_ID"] = "minio"
os.environ["AWS_SECRET_ACCESS_KEY"] = "minio123"
os.environ["MLFLOW_S3_ENDPOINT_URL"] = "http://localhost:9000"

In [ ]:
from workflow.dags.extract import extract_data,connect_minio
from workflow.dags.transform import process_data
from training.dlrm import train_dlrm, download_parquet, test_dlrm_model

In [ ]:
extract_data()
process_data("dlrm")

In [ ]:
train, test = download_parquet(os.environ["DB_MINIO_BUCKET"], "dlrm_train", "dlrm_test")

In [ ]:
train.head(1)

In [ ]:
train_dlrm(
    train,
    test,
    2,
    "movie_recom",
    "http://localhost:8081"
)

In [ ]:
test_dlrm_model()